In [1]:
from scipy.sparse import csr_matrix, block_diag
import cv2
import numpy as np
import pandas as pd

In [2]:
np.set_printoptions(precision=3, floatmode='fixed')

### №17

In [3]:
points = np.random.rand(10, 3)
points[:, 2] += 1

K = np.eye(3)

R1 = np.eye(3)
t1 = np.zeros((3, 1))

R2 = np.eye(3)
t2 = np.array([1, 0 ,0], dtype=np.float32)

points_1, _ = cv2.projectPoints(points, R1, t1, K, None)
points_2, _ = cv2.projectPoints(points, R2, t2, K, None)

E, _ = cv2.findEssentialMat(points_1, points_2, K, cv2.RANSAC)

print("Необходимая матрица E:")
print(E)
print("Сингулярные числа E:")
print(np.linalg.svd(E)[1])

Необходимая матрица E:
[[ 0.078  0.653 -0.234]
 [ 0.274 -0.267 -0.556]
 [-0.099 -0.024  0.213]]
Сингулярные числа E:
[7.071e-01 7.071e-01 2.183e-16]


### №18

In [4]:
R1, R2, T = cv2.decomposeEssentialMat(E)

ones = np.ones(points.shape[0])
homogen_coords = np.hstack((points.reshape(-1, 3), ones[:, np.newaxis]))
positive_counts = []

possible_cases = [np.hstack((R1, T)), np.hstack((R1, -T)), 
                  np.hstack((R2, T)), np.hstack((R2, -T))]

for _, case in enumerate(possible_cases):
    new_points = case @ homogen_coords.T
    positive_z_count = sum(new_points[2] >= 0)
    positive_counts.append(positive_z_count)

max_index = positive_counts.index(max(positive_counts))

print("Поза первой камеры относительно второй [R|T]:")
print(possible_cases[max_index])

Поза первой камеры относительно второй [R|T]:
[[ 0.267 -0.353 -0.897  0.155]
 [-0.393 -0.890  0.234  0.296]
 [-0.880  0.290 -0.376  0.943]]


### №19

In [5]:
image_path = "GOPR01170000.jpg"
image = cv2.imread(image_path)

f_len = 0.64 * 25.4
image_width = image.shape[1]
image_height = image.shape[0]

pixel_size_x = 36 / image_width
pixel_size_y = 24 / image_height

f_len_px = f_len / pixel_size_x
f_len_py = f_len / pixel_size_y

cx = image_width / 2
cy = image_height / 2

K1 = np.array([[4.2581151397691417e+02, 0.0, 4.8175802555527360e+02], 
               [ 0.0, 4.3533255745518812e+02, 2.6743704185634374e+02], 
               [ 0.0, 0.0, 1.0]], dtype=np.float32)

K2 = np.array([[f_len_px, 0, cx],
              [0, f_len_py, cy],
              [0, 0, 1]], dtype=np.float32)

print("Параметры, полученные методом калибрации камеры")
print(K1)
print("Параметры, полученные при предположении")
print(K2)

Параметры, полученные методом калибрации камеры
[[425.812   0.000 481.758]
 [  0.000 435.333 267.437]
 [  0.000   0.000   1.000]]
Параметры, полученные при предположении
[[433.493   0.000 480.000]
 [  0.000 365.760 270.000]
 [  0.000   0.000   1.000]]


* Метод калибрации камеры в OpenCV использует вектор коэффициентов дисторсии.

* Ошибки в предположениях: фокусное расстояние в эквиваленте 35 мм может быть приближенным значением и не соответствовать фактическому фокусному расстоянию, а оптическая ось может не проходить строго через центр изображения.

### №20

In [6]:
np.set_printoptions(precision=0, floatmode='fixed')

In [7]:
n = 10
m = 3 * 10 ** 6

r = pd.read_csv('rhs.csv', header=None)
r = r.values.flatten()

i = np.arange(n).reshape(n, 1)
j = np.arange(n)
C = 1 / (i + j + 1)

i = np.arange(1, n + 1)
j = np.arange(1, m + 1)
I, J = np.meshgrid(i, j, indexing='ij')
D = np.cos(I * J)

Gb = np.array([[1, 0, 1],
              [0, 1, 0],
              [-1, 0, 1]])

Gb = [Gb] * (m // Gb.shape[0])
G = csr_matrix(block_diag(Gb))

Gb_inv = np.linalg.inv(Gb)
G_inv = csr_matrix(block_diag(Gb_inv))

A = C - D @ G_inv @ D.T

a = r[:10]
b = r[10:]

del_c = np.linalg.solve(A, a - D @ G_inv @ b)
del_S = G_inv @ (b - D.T @ del_c)

x = np.concatenate([del_c, del_S])

print("Первые 20 значений вектора x:", x[:20])

Первые 20 значений вектора x: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]
